<a href="https://colab.research.google.com/github/milver/Experiments/blob/main/NewsletterSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the required libraries
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup


In [ ]:
# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

# Variable creds will store the user access token.
# If no valid token found, we will create one.
creds = None

# The file token.pickle contains the user access token.
# Check if it exists
if os.path.exists('token.pickle'):

	# Read the token from the file and store it in the variable creds
	with open('token.pickle', 'rb') as token:
		creds = pickle.load(token)

# If credentials are not available or are invalid, ask the user to log in.
if not creds or not creds.valid:
	if creds and creds.expired and creds.refresh_token:
		creds.refresh(Request())
	else:
		flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
		creds = flow.run_local_server(port=0)

	# Save the access token in token.pickle file for the next run
	with open('token.pickle', 'wb') as token:
		pickle.dump(creds, token)





In [ ]:
import base64
from googleapiclient.discovery import build
from bs4 import BeautifulSoup

def extract_text_with_links(soup):
    """ Extracts text from a BeautifulSoup object and retains hyperlinks in a readable format. """
    for a in soup.find_all('a'):
        href = a.get('href', '')
        if a.text:
            a.replace_with(f"{a.text} [{href}]")
        else:
            a.replace_with(f"[{href}]")
    return soup.get_text()

# Initialize the Gmail API
service = build('gmail', 'v1', credentials=creds)

# Define the query to fetch emails
query = 'label:Newsletters after:2024/04/12'

# Fetch messages from Gmail API
result = service.users().messages().list(maxResults=10, q=query, userId='me').execute()
messages = result.get('messages', [])

# Write output to a file
with open('email_contents.txt', 'w', encoding='utf-8') as file:
    for msg in messages:
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        payload = txt.get('payload', {})
        headers = payload.get('headers', [])

        subject = ''
        sender = ''
        # Extract Subject and Sender from the headers
        for d in headers:
            if d['name'] == 'Subject':
                subject = d['value']
            elif d['name'] == 'From':
                sender = d['value']

        body_text = ''
        # Check if the email has parts (multipart emails)
        if 'parts' in payload:
            for part in payload['parts']:
                mime_type = part.get('mimeType', '')
                data = part['body'].get('data', '')
                if data:
                    data = data.replace("-", "+").replace("_", "/")
                    decoded_data = base64.urlsafe_b64decode(data)
                    if mime_type == 'text/plain':
                        body_text += decoded_data.decode('utf-8')
                    elif mime_type == 'text/html':
                        soup = BeautifulSoup(decoded_data, 'lxml')
                        body_text += extract_text_with_links(soup)

        # Write the formatted email details to the file
        file.write(f"From: {sender}\nSubject: {subject}\nMessage:\n{body_text.strip()}\n")
        file.write("-" * 80 + "\n")  # Separator line
